In [10]:
%addjar --magic http://www.brunelvis.org/jar/spark-kernel-brunel-all-1.1.jar

Using cached version of spark-kernel-brunel-all-1.1.jar


In [11]:
val sql = sqlContext
import sql.implicits._
import java.sql.Date
import java.time._, format._
import org.apache.spark.sql.functions._

In [99]:
val input = sc.textFile("input/transactions.tab")
val tabSeparated = input.map(_.split("\\t"))

In [50]:
case class Transaction(
  date: Date,
  amount: BigDecimal,
  comment: String  
)

In [100]:
val transactions = {
    tabSeparated.map { row =>
        val formatter = DateTimeFormatter.ofPattern("yyyyMMdd")
        Transaction(
            date = Date valueOf LocalDate.parse(row(2), formatter),
            amount = BigDecimal(row(6).replace(",", ".")),
            comment = row(7)
        )
    }
    .toDS

}

def isAtmLoad(t: Transaction) = t.comment contains "STORTING"

val df = transactions.filter(!isAtmLoad(_)).toDF.registerTempTable("transactions")

In [101]:
transactions.count()

476

In [102]:
%%sql select sum(amount) as saldo from transactions

+--------------------+
|               saldo|
+--------------------+
|9086.050000000000...|
+--------------------+

